In [1]:
import os
import angr, claripy


filePath = '/home/nahid/reverse/binaries/fauxware'
proj = angr.Project(filePath, load_options={"auto_load_libs": False})

In [2]:


cfg = proj.analyses.CFGEmulated(keep_state=True,
                             state_add_options=angr.sim_options.refs,
                             context_sensitivity_level=2)



WARNING  | 2023-06-06 22:10:02,109 | angr.storage.memory_mixins.default_filler_mixin | The program is accessing register with an unspecified value. This could indicate unwanted behavior.
WARNING  | 2023-06-06 22:10:02,110 | angr.storage.memory_mixins.default_filler_mixin | angr will cope with this by generating an unconstrained symbolic variable and continuing. You can resolve this by:
WARNING  | 2023-06-06 22:10:02,110 | angr.storage.memory_mixins.default_filler_mixin | 1) setting a value to the initial state
WARNING  | 2023-06-06 22:10:02,110 | angr.storage.memory_mixins.default_filler_mixin | 2) adding the state option ZERO_FILL_UNCONSTRAINED_{MEMORY,REGISTERS}, to make unknown regions hold null
WARNING  | 2023-06-06 22:10:02,111 | angr.storage.memory_mixins.default_filler_mixin | 3) adding the state option SYMBOL_FILL_UNCONSTRAINED_{MEMORY,REGISTERS}, to suppress these messages.
WARNING  | 2023-06-06 22:10:02,111 | angr.storage.memory_mixins.default_filler_mixin | Filling register 

In [3]:
# Generate the control dependence graph
cdg = proj.analyses.CDG(cfg)

In [4]:
# Build the data dependence graph. It might take a while. Be patient!
ddg = proj.analyses.DDG(cfg)


# See where we wanna go... let's go to the exit() call, which is modeled as a
# SimProcedure.
target_func = cfg.kb.functions.function(name="exit")
print(target_func)
# We need the CFGNode instance
target_node = cfg.get_any_node(target_func.addr)

Function exit [4195696]
  Syscall: False
  SP difference: 0
  Has return: False
  Returning: False
  Alignment: False
  Arguments: reg: [], stack: []
  Blocks: [0x400570]
  Calling convention: None


In [5]:
print(target_node)

<CFGENode 0x400570[6]>


In [6]:
# Let's get a BackwardSlice out of them!
# ``targets`` is a list of objects, where each one is either a CodeLocation
# object, or a tuple of CFGNode instance and a statement ID. Setting statement
# ID to -1 means the very beginning of that CFGNode. A SimProcedure does not
# have any statement, so you should always specify -1 for it.
print('Target Node: ', target_node)
bs = proj.analyses.BackwardSlice(cfg, cdg=cdg, ddg=ddg, targets=[ (target_node, -1) ])



Target Node:  <CFGENode 0x400570[6]>


In [21]:
from angrutils import *
# print(bs.kb.cfgs.cfgs.values())
print(bs.runs_in_slice)
print('cfg_nodes_in_slice',bs.cfg_nodes_in_slice)
print('chosen_statements',bs.chosen_statements)
print('\n\n chosen_exits',bs.chosen_exits)
acfg = bs.annotated_cfg()
print(dir( acfg))


None
cfg_nodes_in_slice None
chosen_statements defaultdict(<class 'set'>, {4196093: {-2}, 4196115: {-2}, 4195696: {-2}, 4196275: {30, -2}, 4196297: {-2}, 4196040: {-2}, 4196059: {-2}, 4196070: {-2}, 4195982: {17, -2}, 4195986: {-2}, 4196075: {-2}, 4195940: {-2}, 4195664: {-2}, 7340064: {-2}, 4195918: {13, -2}, 4195756: {19, -2}, 4195904: {21, -2}, 4195774: {-2}, 4195561: {-2}, 4195993: {-2}, 4196015: {-2}})


 chosen_exits defaultdict(<class 'list'>, {4196093: [(-2, 4196115)], 4196275: [(30, 4196297)], 4196040: [(-2, 4196059)], 4195982: [(17, 4195986), (-2, 4195993)], 4195940: [(-2, 4195982), (-2, 4195664)], 4195904: [(-2, 4195918)], 4195756: [(19, 4195774)], 4195993: [(-2, 4196015)], 4196115: [(-2, None)], 4196297: [(-2, None)], 4196059: [(-2, None)], 4195986: [(-2, None)], 4195664: [(-2, None)], 4195918: [(-2, None)], 4195774: [(-2, None)], 4196015: [(-2, None)], None: [(-2, None)], 4195561: [], 4195696: []})
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '_

In [11]:
plot_cfg(cfg, "faux", asminst = True, remove_imports = True, remove_path_terminator=True)  

In [12]:
bs.dbg_repr()

'BackwardSlice (to [(<CFGENode 0x400570[6]>, -1)])\n21 SimRuns in program slice, displaying 10.\n0x4004e9\n-[  0] ------ IMark(0x4004e9, 5, 0) ------\n-[  1] t3 = GET:I64(offset=48)\n-[  2] t2 = Sub64(t3,0x0000000000000008)\n-[  3] PUT(offset=48) = t2\n-[  4] STle(t2) = 0x00000000004004ee\n-[  5] t4 = Sub64(t2,0x0000000000000080)\n-[  6] ====== AbiHint(0xt4, 128, 0x0000000000400640) ======\nChosen exits: \n0x400550\n-[  0] ------ IMark(0x400550, 6, 0) ------\n-[  1] t2 = LDle:I64(0x0000000000601020)\nChosen exits: default\n0x400570\n-[  0] ------ IMark(0x400570, 6, 0) ------\n-[  1] t2 = LDle:I64(0x0000000000601030)\nChosen exits: \n0x4005ac\n-[  0] ------ IMark(0x4005ac, 4, 0) ------\n-[  1] t2 = GET:I64(offset=48)\n-[  2] t0 = Sub64(t2,0x0000000000000008)\n-[  3] PUT(offset=48) = t0\n-[  4] PUT(offset=184) = 0x00000000004005b0\n-[  5] ------ IMark(0x4005b0, 7, 0) ------\n-[  6] t7 = LDle:I64(0x0000000000600fe0)\n-[  7] PUT(offset=16) = t7\n-[  8] ------ IMark(0x4005b7, 3, 0) ------\n

In [13]:
# import networkx as nx
# import matplotlib.pyplot as plt
# # Need to create a layout when doing
# # separate calls to draw nodes and edges
# G= bs.cfg_nodes_in_slice()
# print(G)
# pos = nx.spring_layout(G)
# nx.draw_networkx_nodes(G, pos, cmap=plt.get_cmap('jet'), 
#                        node_color = values, node_size = 500)
# nx.draw_networkx_labels(G, pos)
# nx.draw_networkx_edges(G, pos, edgelist=red_edges, edge_color='r', arrows=True)
# nx.draw_networkx_edges(G, pos, edgelist=black_edges, arrows=False)
# plt.show()